In [94]:
# Load libraries
import numpy as np

In [95]:
# Set seed
np.random.seed(10)

In [96]:
# Doc params
V = 50
N = 800
K = 3
M = 2

In [97]:
# Set true params
alpha_true = np.array([10, 1, 2])
beta_true = np.random.randint(1, 10, V)

In [98]:
# Generate data
phi_true = np.zeros((K, V))
for k in range(K):
    phi_true[k, :] = np.random.dirichlet(beta_true)

In [99]:
theta_true = np.zeros((M, K))
for m in range(M):
    theta_true[m,:] = np.random.dirichlet(alpha_true)

In [100]:
doc_lens = np.random.randint(100, N, M)
z_true = {}
w = {}
for m in range(M):
    z_true[m] = []
    w[m] = []
    for n in range(doc_lens[m]):
        z_true[m].extend(np.nonzero(np.random.multinomial(1, theta_true[m,:]))[0])
        w[m].extend(np.nonzero(np.random.multinomial(1, phi_true[z_true[m][n], :]))[0])

In [101]:
#z_true = np.zeros((M, N), dtype = "int")
#w = np.zeros((M, N), dtype = "int")
#for m in range(M):
#    for n in range(N):
#        z_true[m, n] = np.nonzero(np.random.multinomial(1, theta_true[m,:]))[0]
#        w[m, n] = np.nonzero(np.random.multinomial(1, phi_true[z_true[m, n], :]))[0]
len(w)

2

In [102]:
# Set initial z randomly
z = {}
for m in range(M):
    z[m] = []
    for n in range(doc_lens[m]):
        z[m].extend(np.nonzero(np.random.multinomial(1, np.ones(K)/K))[0])

In [103]:
# Create count matrices
A = np.zeros((M, K))
for m in range(M):
    for k in range(K):
        A[m, k] = sum(np.array(z[m]) == k)

In [104]:
B = np.zeros((K, V))
for m in range(M):
    for n in range(doc_lens[m]):
        B[z[m][n], w[m][n]] += 1

In [105]:
C = np.zeros(K)
for m in range(M):
    for n in range(doc_lens[m]):
        C[z[m][n]] += 1

In [106]:
# Set initial z randomly
#z = np.zeros((M, N), dtype = "int")
#for m in range(M):
#    for n in range(N):
#        z[m, n] = np.random.binomial(1, 0.5)

In [107]:
# Create count matrices
#A = np.zeros((M, K))
#for m in range(M):
#    for k in range(K):
#        A[m, k] = sum(z[m, :] == k)

In [108]:
#B = np.zeros((K, V))
#for m in range(M):
#    for n in range(N):
#        B[z[m, n], w[m, n]] += 1

In [109]:
#C = np.zeros(K)
#for m in range(M):
#    for n in range(N):
#        C[z[m, n]] += 1

In [110]:
# set hyperparameters alpha and beta
alpha = np.ones(K)
beta = np.ones(V)
#alpha = alpha_true
#beta = beta_true

In [112]:
# start gibbs sampler
num_iter = 10000
for i in range(num_iter):
    for m in range(M):
        for n in range(doc_lens[m]):
            A[m, z[m][n]] -= 1  # Decrement N1
            B[z[m][n], w[m][n]] -= 1 # Decrement N2
            C[z[m][n]] -= 1 # Decrement N3
            p = np.zeros(K)
            for k in range(K):
                p[k] = (A[m, k] + alpha[k])*((B[k, w[m][n]] + beta[w[m][n]])/(C[k] + sum(beta)))
            p /= sum(p) # This is actually doing k divisions... might be a modest speed up but we can parallelize easily with numba
            z[m][n] = np.nonzero(np.random.multinomial(1, p))[0][0]
            A[m, z[m][n]] += 1 # Increment N1
            B[z[m][n], w[m][n]] += 1 # Increment N2
            C[z[m][n]] += 1 # Increment N3

SyntaxError: invalid syntax (<ipython-input-112-5db361745e8f>, line 3)

In [ ]:
# start gibbs sampler
#num_iter = 10000
#for i in range(num_iter):
#    for m in range(M):
#        for n in range(N):
#            A[m, z[m, n]] -= 1
#            B[z[m, n], w[m, n]] -= 1
#            C[z[m, n]] -= 1
#            p = np.zeros(K)
#            for k in range(K):
#                p[k] = (A[m, k] + alpha[k])*((B[k, w[m, n]] + beta[w[m, n]])/(C[k] + sum(beta)))
#            p /= sum(p)
#            z[m, n] = np.nonzero(np.random.multinomial(1, p))[0]
#            A[m, z[m, n]] += 1
#            B[z[m, n], w[m, n]] += 1
#            C[z[m, n]] += 1

In [113]:
theta = np.zeros((M, K))
for m in range(M):
    for k in range(K):
        theta[m , k] = (A[m, k] + alpha[k])/(sum(A[m,:]) + sum(alpha))
np.round(theta, 2)

array([[ 0.48,  0.35,  0.17],
       [ 0.59,  0.38,  0.03]])

In [114]:
np.round(theta_true, 2)

array([[ 0.83,  0.04,  0.13],
       [ 0.71,  0.11,  0.19]])